In [51]:
import pandas as pd
import numpy as np
import math
import json
import pprint
import requests
from sqlalchemy import create_engine, inspect
from sqlalchemy import Column, Integer, String, Float

In [64]:
USER_01_MAPPING = 'resources/raw_data_next_gen/admin_users.csv'
PROJECT_01_MAPPING = 'resources/raw_data_next_gen/admin_projects.csv'
ISSUE_01_MAPPING = 'resources/raw_data_next_gen/issues_issues.csv'
COMPANIES_01_MAPPING = 'resources/raw_data_next_gen/admin_companies.csv'
PROJECT_COMPANIES_01_MAPPING = 'resources/raw_data_next_gen/admin_project_companies.csv'
BUSINESS_01_MAPPING = 'resources/raw_data_next_gen/admin_business_units.csv'
ISSUETYPES_01_MAPPING = 'resources/raw_data_next_gen/issues_issue_types.csv'
ISSUESUBTYPES_01_MAPPING = 'resources/raw_data_next_gen/issues_issue_subtypes.csv'

##############

USER_02_MAPPING = 'resources/raw_data_old_gen/20200330085907_FieldDataHarvest_Users.csv'
PROJECT_02_MAPPING = 'resources/raw_data_old_gen/20200330085907_FieldDataHarvest_Projects.csv'
ISSUE_02_MAPPING = 'resources/raw_data_old_gen/20200330085907_FieldDataHarvest_Issues.csv'

In [65]:
U1_DF = pd.read_csv(USER_01_MAPPING, encoding='utf-8', low_memory=False)
P1_DF = pd.read_csv(PROJECT_01_MAPPING, encoding='utf-8', low_memory=False)
I1_DF = pd.read_csv(ISSUE_01_MAPPING, encoding='latin1', low_memory=False)
C1_DF = pd.read_csv(COMPANIES_01_MAPPING, encoding='latin1', low_memory=False)
PC1_DF = pd.read_csv(PROJECT_COMPANIES_01_MAPPING, encoding='latin1', low_memory=False)
B1_DF = pd.read_csv(BUSINESS_01_MAPPING, encoding='latin1', low_memory=False)
IT1_DF = pd.read_csv(ISSUETYPES_01_MAPPING, encoding='latin1', low_memory=False)
IST1_DF = pd.read_csv(ISSUESUBTYPES_01_MAPPING, encoding='latin1', low_memory=False)

U2_DF = pd.read_csv(USER_02_MAPPING, encoding='utf-8', low_memory=False)
P2_DF = pd.read_csv(PROJECT_02_MAPPING, encoding='utf-8', low_memory=False)
I2_DF = pd.read_csv(ISSUE_02_MAPPING, encoding='latin1', low_memory=False)


In [4]:
U1_DF['domain']= U1_DF['email'].str.split('@').str[1]
len(U1_DF['domain'].unique())
domain_A = U1_DF['domain'].unique()

In [102]:
#domain_A[1].split('.')[0]

In [103]:
# temp_DF = U1_DF.loc[U1_DF['domain']==domain_A[16]]
# temp_DF
# LIST_D=[]
# LIST_D = temp_DF.loc[temp_DF['default_company_id'].notnull()]['default_company_id'] 
#res = max(set(LIST_D), key = LIST_D.count) 

In [104]:
# x = LIST_D.value_counts().idxmax()

In [118]:

r = 0
#for index, row in U1_DF.iterrows():

for i in range (1, len(domain_A)):
    LIST_D=[]
    r = r + 1
    temp_DF = U1_DF.loc[U1_DF['domain']==domain_A[i]]
        
        #LIST_D = temp_DF.loc[temp_DF['default_company_id'].notnull()]
        
    LIST_D = temp_DF.loc[temp_DF['default_company_id'].notnull()]['default_company_id']
    if len(LIST_D) > 0:
        comp_id = LIST_D.value_counts().idxmax()
#         print("LIST of DOMAINS....",LIST_D)
        for index, row in temp_DF.iterrows():
            rowIndex = (index)
        
            if domain_A[i]>'':
                d_value = str(domain_A[i])
                if (d_value.split('.')[0]) not in ['gmail','outlook','yahoo','msn','swbell','comcast','icloud','charter']:
        
#                     print (r,'.....', d_value, '----------GROUP----------------------------')
#                     for index, row in temp_DF.iterrows():
                    x1 = (row['domain'])
                    x2 = (row['default_company_id'])
#                     print(x2)
                    
                    if type(x2) == float:
                        U1_DF.loc[rowIndex, 'default_company_id'] = comp_id
                        print(rowIndex, x1,'.......',x2)
        
print ('NEW GROUP-----------------------------------------------------')

NEW GROUP-----------------------------------------------------


In [95]:
#######################################################
P1_DF.rename(columns = {"id" : "project_id"}, inplace=True)
PC1_DF.rename(columns = {"company_id" : "default_company_id"}, inplace=True)
I1_DF.rename(columns = {"bim360_project_id":"project_id"}, inplace=True)
C1_DF.rename(columns = {"id" : "default_company_id"}, inplace=True)

In [122]:
C1_DF.to_csv('cleaned_companies.csv')

In [20]:
U1_DF.count()

id                             9341
autodesk_id                    6769
bim360_account_id              9341
email                          9303
name                           9341
first_name                     9341
last_name                      9341
address_line1                   253
address_line2                    55
city                            314
state_or_province               436
postal_code                     271
country                        9327
last_sign_in                   3101
phone                           324
job_title                       444
access_level_account_admin     9341
access_level_project_admin     9341
access_level_project_member    9341
access_level_executive         9341
default_role_id                 919
default_company_id             3617
dtype: int64

In [12]:
MERGED_PI_NEW = pd.merge(P1_DF, I1_DF, on ='project_id', how='inner')

#MERGED_UC_NEW = pd.merge(U1_DF, C1_DF, on ='default_company_id', how='right')
#MERGED_PUC_NEW = pd.merge(PC1_DF, MERGED_UC_NEW, on ='default_company_id', how='inner')

In [78]:
#MERGED_PI_NEW.count()

In [34]:
# Working_OLD_DataFrame
W_NEW_PI_DF = MERGED_PI_NEW[['project_id',
                   'name',
                   'title',
                   'description',
                   'type_id',
                   'subtype_id',
                   'assignee_id',
                   'job_number',
                   'type',
                   'value',
                   'city',
                   'state_or_province',
                   'postal_code']].copy()

In [35]:
W_NEW_PI_DF.rename(columns = {"type_id" : "issue_type_id"}, inplace=True)
W_NEW_PI_DF.head()

,project_id,name,title,description,issue_type_id,subtype_id,assignee_id,job_number,type,value,city,state_or_province,postal_code
0,373096cf-bb05-4c80-8cfe-aa4f978ef5d0,00000_Mercy_Hospital_Springfield_Pharmacy,Scan_043_044_High_Low,NaN,fcad3025-f61e-4b23-9aca-fe550600b014,51ee74cd-a8fc-4acf-b665-1e6097c123bd,200810021927646,Unknown,Hospital,1000000.0,Springfield,MO,65804
1,373096cf-bb05-4c80-8cfe-aa4f978ef5d0,00000_Mercy_Hospital_Springfield_Pharmacy,Scan_007_Low,NaN,fcad3025-f61e-4b23-9aca-fe550600b014,51ee74cd-a8fc-4acf-b665-1e6097c123bd,200810021927646,Unknown,Hospital,1000000.0,Springfield,MO,65804
2,373096cf-bb05-4c80-8cfe-aa4f978ef5d0,00000_Mercy_Hospital_Springfield_Pharmacy,Scan_019_020_High_Low,NaN,fcad3025-f61e-4b23-9aca-fe550600b014,51ee74cd-a8fc-4acf-b665-1e6097c123bd,200810021927646,Unknown,Hospital,1000000.0,Springfield,MO,65804
3,373096cf-bb05-4c80-8cfe-aa4f978ef5d0,00000_Mercy_Hospital_Springfield_Pharmacy,Scan_021_022_High_Low,NaN,fcad3025-f61e-4b23-9aca-fe550600b014,51ee74cd-a8fc-4acf-b665-1e6097c123bd,200810021927646,Unknown,Hospital,1000000.0,Springfield,MO,65804
4,373096cf-bb05-4c80-8cfe-aa4f978ef5d0,00000_Mercy_Hospital_Springfield_Pharmacy,Scan_041_042_High_Low,NaN,fcad3025-f61e-4b23-9aca-fe550600b014,51ee74cd-a8fc-4acf-b665-1e6097c123bd,200810021927646,Unknown,Hospital,1000000.0,Springfield,MO,65804


In [66]:
IT1_DF.count()

issue_type_id        3638
bim360_account_id    3638
bim360_project_id    3638
issue_type           3638
is_active            3638
created_by           3266
created_at           3638
updated_by           3290
updated_at           3638
deleted_by              0
deleted_at              0
dtype: int64

In [58]:
MERGED_PIIT_NEW = pd.merge(W_NEW_PI_DF, IT1_DF, on ='issue_type_id', how='inner')

In [59]:
MERGED_PIIT_NEW.head()

,project_id,name,title,description,issue_type_id,subtype_id,assignee_id,job_number,type,value,...,bim360_account_id,bim360_project_id,issue_type,is_active,created_by,created_at,updated_by,updated_at,deleted_by,deleted_at
0,373096cf-bb05-4c80-8cfe-aa4f978ef5d0,00000_Mercy_Hospital_Springfield_Pharmacy,Scan_043_044_High_Low,NaN,fcad3025-f61e-4b23-9aca-fe550600b014,51ee74cd-a8fc-4acf-b665-1e6097c123bd,200810021927646,Unknown,Hospital,1000000.0,...,49df69ab-c266-4d49-951c-031b16f7a31f,373096cf-bb05-4c80-8cfe-aa4f978ef5d0,Observation,t,System,2019-05-15 20:05:30.04,System,2019-05-15 20:05:30.04,NaN,NaN
1,373096cf-bb05-4c80-8cfe-aa4f978ef5d0,00000_Mercy_Hospital_Springfield_Pharmacy,Scan_007_Low,NaN,fcad3025-f61e-4b23-9aca-fe550600b014,51ee74cd-a8fc-4acf-b665-1e6097c123bd,200810021927646,Unknown,Hospital,1000000.0,...,49df69ab-c266-4d49-951c-031b16f7a31f,373096cf-bb05-4c80-8cfe-aa4f978ef5d0,Observation,t,System,2019-05-15 20:05:30.04,System,2019-05-15 20:05:30.04,NaN,NaN
2,373096cf-bb05-4c80-8cfe-aa4f978ef5d0,00000_Mercy_Hospital_Springfield_Pharmacy,Scan_019_020_High_Low,NaN,fcad3025-f61e-4b23-9aca-fe550600b014,51ee74cd-a8fc-4acf-b665-1e6097c123bd,200810021927646,Unknown,Hospital,1000000.0,...,49df69ab-c266-4d49-951c-031b16f7a31f,373096cf-bb05-4c80-8cfe-aa4f978ef5d0,Observation,t,System,2019-05-15 20:05:30.04,System,2019-05-15 20:05:30.04,NaN,NaN
3,373096cf-bb05-4c80-8cfe-aa4f978ef5d0,00000_Mercy_Hospital_Springfield_Pharmacy,Scan_021_022_High_Low,NaN,fcad3025-f61e-4b23-9aca-fe550600b014,51ee74cd-a8fc-4acf-b665-1e6097c123bd,200810021927646,Unknown,Hospital,1000000.0,...,49df69ab-c266-4d49-951c-031b16f7a31f,373096cf-bb05-4c80-8cfe-aa4f978ef5d0,Observation,t,System,2019-05-15 20:05:30.04,System,2019-05-15 20:05:30.04,NaN,NaN
4,373096cf-bb05-4c80-8cfe-aa4f978ef5d0,00000_Mercy_Hospital_Springfield_Pharmacy,Scan_041_042_High_Low,NaN,fcad3025-f61e-4b23-9aca-fe550600b014,51ee74cd-a8fc-4acf-b665-1e6097c123bd,200810021927646,Unknown,Hospital,1000000.0,...,49df69ab-c266-4d49-951c-031b16f7a31f,373096cf-bb05-4c80-8cfe-aa4f978ef5d0,Observation,t,System,2019-05-15 20:05:30.04,System,2019-05-15 20:05:30.04,NaN,NaN


In [60]:
MERGED_PIIT_NEW.drop(['is_active','created_by','created_at','updated_by','updated_at','bim360_account_id','bim360_project_id','deleted_by','deleted_at'],inplace=True, axis=1)
MERGED_PIIT_NEW.count()

project_id           59084
name                 59084
title                59084
description          23200
issue_type_id        59084
subtype_id           59084
assignee_id          52446
job_number           57085
type                 59084
value                57593
city                 42841
state_or_province    42592
postal_code          42401
issue_type           59084
dtype: int64

In [61]:
MERGED_PIIT_NEW.rename(columns = {"subtype_id" : "issue_subtype_id"}, inplace=True)



In [71]:
len(IST1_DF['issue_type_id'].unique())

3638

In [111]:
#MERGED_PIIT_NEW.count()
MERGED_PIIST_NEW = pd.merge(MERGED_PIIT_NEW, IST1_DF, on ='issue_subtype_id', how='inner')

In [112]:
MERGED_PIIST_NEW.drop(['issue_type_id_x',
                       'issue_subtype_id',
                       'issue_type_id_y',
                       'is_active',
                       'created_by',
                       'created_at',
                       'updated_by',
                       'updated_at',
                       'deleted_by',
                       'deleted_at',
                       'bim360_account_id',
                       'bim360_project_id'],inplace=True, axis=1)

In [113]:
MERGED_PIIST_NEW.count()

project_id           59084
name                 59084
title                59084
description          23200
assignee_id          52446
job_number           57085
type                 59084
value                57593
city                 42841
state_or_province    42592
postal_code          42401
issue_type           59084
issue_subtype        59084
dtype: int64

In [80]:
#MERGED_PUC_NEW.head()
MERGED_PIIST_NEW.to_csv('NEW_PIIST.csv')

#MERGED_PUC_NEW.rename(columns = {"id" : "project_id"}, inplace=True)

In [11]:
#MERGED_PI_NEW.head()

In [12]:
#MERGED_PIUC_NEW = pd.merge(P1_DF,MERGED_PUC_NEW, on ='project_id', how = 'right')

In [8]:
PC1_DF.count()
len(PC1_DF['project_id'].unique())

773

In [14]:
type(MERGED_PUC_NEW)

pandas.core.frame.DataFrame

In [9]:
len(MERGED_PUC_NEW['email'].unique())

3577

In [15]:
MERGED_PUC_NEW.head()

,project_id,autodesk_id,bim360_account_id_x,email,name_x,first_name,last_name,address_line1_x,address_line2_x,city_x,...,description,erp_id,tax_id,status,created_at,project_size,user_size,custom_properties,project_id,bim360_account_id
0,00727ac6-e54e-4cdf-ba01-68a7f532a1b1,8G6GKJXN5UM5,49df69ab-c266-4d49-951c-031b16f7a31f,djokela@vortexchicago.com,Danielle Jokela,Danielle,Jokela,NaN,NaN,NaN,...,Vortex Commercial Flooring,NaN,NaN,NaN,2015-08-27 02:38:53.855536,5.0,0.0,NaN,47a45a6d-33c7-45b5-a648-e2b6e03377ce,49df69ab-c266-4d49-951c-031b16f7a31f
1,00727ac6-e54e-4cdf-ba01-68a7f532a1b1,8G6GKJXN5UM5,49df69ab-c266-4d49-951c-031b16f7a31f,djokela@vortexchicago.com,Danielle Jokela,Danielle,Jokela,NaN,NaN,NaN,...,Vortex Commercial Flooring,NaN,NaN,NaN,2015-08-27 02:38:53.855536,5.0,0.0,NaN,41eadd13-efd1-4de5-adc9-e7e659608d7e,49df69ab-c266-4d49-951c-031b16f7a31f
2,00727ac6-e54e-4cdf-ba01-68a7f532a1b1,8G6GKJXN5UM5,49df69ab-c266-4d49-951c-031b16f7a31f,djokela@vortexchicago.com,Danielle Jokela,Danielle,Jokela,NaN,NaN,NaN,...,Vortex Commercial Flooring,NaN,NaN,NaN,2015-08-27 02:38:53.855536,5.0,0.0,NaN,03a92e47-f17b-44e1-a326-d29700222caa,49df69ab-c266-4d49-951c-031b16f7a31f
3,00727ac6-e54e-4cdf-ba01-68a7f532a1b1,8G6GKJXN5UM5,49df69ab-c266-4d49-951c-031b16f7a31f,djokela@vortexchicago.com,Danielle Jokela,Danielle,Jokela,NaN,NaN,NaN,...,Vortex Commercial Flooring,NaN,NaN,NaN,2015-08-27 02:38:53.855536,5.0,0.0,NaN,dc5185e7-f10b-47f4-b540-73c0f5d32c5c,49df69ab-c266-4d49-951c-031b16f7a31f
4,00727ac6-e54e-4cdf-ba01-68a7f532a1b1,8G6GKJXN5UM5,49df69ab-c266-4d49-951c-031b16f7a31f,djokela@vortexchicago.com,Danielle Jokela,Danielle,Jokela,NaN,NaN,NaN,...,Vortex Commercial Flooring,NaN,NaN,NaN,2015-08-27 02:38:53.855536,5.0,0.0,NaN,c3c3c797-6faa-4a95-a961-e760eb10bdee,49df69ab-c266-4d49-951c-031b16f7a31f


In [11]:
MERGED_UC_NEW.count()
#MERGED_PIUC_NEW = pd.merge(MERGED_PUC_NEW,PC1_DF, on ='project_id', how = 'right')

id                             3617
autodesk_id                    3221
bim360_account_id_x            3617
email                          3617
name_x                         3617
first_name                     3617
last_name                      3617
address_line1_x                 120
address_line2_x                  33
city_x                          159
state_or_province_x             239
postal_code_x                   131
country_x                      3615
last_sign_in                   2693
phone_x                         159
job_title                       272
access_level_account_admin     3617
access_level_project_admin     3617
access_level_project_member    3617
access_level_executive         3617
default_role_id                 792
default_company_id             3617
bim360_account_id_y            3617
name_y                         3617
trade                          2320
category                       2373
address_line1_y                1060
address_line2_y             

In [ ]:
#PC_s=PC.sort_values(by=['project_id'])

In [ ]:
#PC_s.head(50)

In [91]:
OLD_PI_DF = pd.merge(P2_DF, I2_DF, on='project_id', how='inner')


In [92]:
OLD_PI_DF.rename(columns = {"id":"project_id"}, inplace=True)
OLD_PI_DF.drop(['business_unit_x',
                'status_x',
                'checklists',
                'equipment',
                'tasks',
                'dailyUpdates',
                'photos',
                'users',
                'identifier',
                'status_y','priority',
                'full_description',
                'location_detail',
                'source_type',
                'parent_source_type',
                'created_by',
                'area',
                'created_at',
                'updated_at',
                'created_from',
                'due_date',
                'closed_at',
                'root_cause',
                'hq_identifier_y',
                'business_unit_y'],inplace=True, axis=1)

In [93]:
OLD_PI_DF.count()

hq_identifier_x     101605
project_id          101605
name                101605
files               101605
issues              101605
issue_id            101605
description         101605
issue_type          101605
company_assigned     96657
dtype: int64

In [100]:
OLD_PI_DF.rename(columns = {"hq_identifier_x":"project_id","project_id":"old_project_id"}, inplace=True)

In [115]:
MERGED_PI_OLD2NEW = pd.merge(OLD_PI_DF, P1_DF, on ='project_id', how='inner')

In [116]:
MERGED_PI_OLD2NEW.count()
MERGED_PI_OLD2NEW.drop(['files',
                        'bim360_account_id',
                        'address_line1',
                        'address_line2',
                        'start_date',
                        'end_date',
                        'currency',
                        'timezone',
                        'construction_type',
                        'contract_type',
                        'business_unit_id',
                        'last_sign_in',
                        'created_at',
                        'country',
                        ],inplace=True, axis=1)

In [123]:
MERGED_PI_OLD2NEW.count()


project_id           101605
old_project_id       101605
name_x               101605
issues               101605
issue_id             101605
description          101605
issue_type           101605
company_assigned      96657
name_y               101605
type                 101605
value                101359
status               101605
job_number            51245
city                  81746
state_or_province     82679
postal_code           77670
dtype: int64

In [124]:
MERGED_PI_OLD2NEW.head()

,project_id,old_project_id,name_x,issues,issue_id,description,issue_type,company_assigned,name_y,type,value,status,job_number,city,state_or_province,postal_code
0,01fa4f3a-9969-4069-a5b3-72fb630a0143,6fbfddc0-b8bb-4b27-be7f-751a21c701d0,Gestamp,476,00bc0784-ed90-41fe-97d2-a71831f0e1dd,Remove nails around dock angles.,Punch List : Pre-Punch List,Concrete Strategies LLC,Gestamp,Manufacturing / Factory,20549347.0,inactive,102601,McCalla,AL,35111
1,01fa4f3a-9969-4069-a5b3-72fb630a0143,6fbfddc0-b8bb-4b27-be7f-751a21c701d0,Gestamp,476,00d3a27e-8127-4dec-ba44-74da29274ac8,"Grind in a 3/4"" chamfer at all exterior corner...",Punch List : Pre-Punch List,Concrete Strategies LLC,Gestamp,Manufacturing / Factory,20549347.0,inactive,102601,McCalla,AL,35111
2,01fa4f3a-9969-4069-a5b3-72fb630a0143,6fbfddc0-b8bb-4b27-be7f-751a21c701d0,Gestamp,476,00dfc82b-7220-4345-93ea-d0ff4ba06d03,Install missing switch and data outlets,Punch List : Architect Punch List,"Ace Electric, Inc.",Gestamp,Manufacturing / Factory,20549347.0,inactive,102601,McCalla,AL,35111
3,01fa4f3a-9969-4069-a5b3-72fb630a0143,6fbfddc0-b8bb-4b27-be7f-751a21c701d0,Gestamp,476,01a36881-f79f-4308-9fa1-55b48f7c58f2,Install cover plate on light switch,Punch List : Architect Punch List,"Ace Electric, Inc.",Gestamp,Manufacturing / Factory,20549347.0,inactive,102601,McCalla,AL,35111
4,01fa4f3a-9969-4069-a5b3-72fb630a0143,6fbfddc0-b8bb-4b27-be7f-751a21c701d0,Gestamp,476,01cebbe8-8a06-489f-a724-bb1b94844b0b,Install base,Punch List : Architect Punch List,Spectra Contract Flooring,Gestamp,Manufacturing / Factory,20549347.0,inactive,102601,McCalla,AL,35111


In [145]:
###################################################
#### TO CLEAN
MERGED_PI_OLD2NEW.head()
PI_OLD2NEW_DF = MERGED_PI_OLD2NEW[['project_id',
                                  'name_x',
                                  'job_number',
                                  'type',
                                  'value',
                                  'city',
                                  'state_or_province',
                                  'postal_code',
                                  'issue_type',
                                  'description',
                                  'company_assigned']].copy()

In [138]:
PI_OLD2NEW_DF.count()


project_id           101605
name_x               101605
job_number            51245
type                 101605
value                101359
city                  81746
state_or_province     82679
postal_code           77670
issue_type           101605
description          101605
company_assigned      96657
dtype: int64

In [141]:
PIIST_NEW_DF = MERGED_PIIST_NEW [['project_id',
                                  'name',
                                  'job_number',
                                  'type',
                                  'value',
                                  'city',
                                  'state_or_province',
                                  'postal_code',
                                  'issue_type',
                                  'issue_subtype',
                                  'title',
                                  'description',
                                  'assignee_id']].copy()

In [143]:
PIIST_NEW_DF.count()


project_id           59084
name                 59084
job_number           57085
type                 59084
value                57593
city                 42841
state_or_province    42592
postal_code          42401
issue_type           59084
issue_subtype        59084
title                59084
description          23200
assignee_id          52446
dtype: int64

In [15]:
#I2_DF.head()

In [16]:
#MERGED_NEW = pd.merge(P1_DF, I1_DF, on='project_id', how='inner')

In [17]:
MERGED_OLD.to_csv('old_clean.csv')

In [23]:
I1_DF.rename(columns = {"bim360_project_id" : "project_id"}, inplace=True)

In [19]:
#I1_DF.dtypes

In [21]:
#P1_DF.dtypes

In [15]:
#MERGED_NEW.count()

In [24]:
MERGED_NEW.to_csv('new_clean.csv')

In [25]:
#P1_DF.head()

In [27]:
#P1_DF.count()

In [28]:
MERGED_OLD.rename(columns = {"project_id" : "project_id_classic"}, inplace=True)
MERGED_OLD.rename(columns = {"hq_identifier_x" : "project_id"}, inplace=True)

In [29]:
MERGED_OLD_NEXT = pd.merge(MERGED_OLD, P1_DF, on='project_id', how='inner')

In [31]:
#MERGED_OLD_NEXT.head()

In [54]:
#MERGED_OLD.head()

In [50]:
#P2_DF.head()

In [56]:
#MERGED_OLD_NEXT.count()

In [32]:
MERGED_OLD_NEXT.to_csv('old_next_clean.csv')

In [35]:
### FINAL MERGE PROJECT ISSUES
OLD_MAPPING = 'old_next_clean.csv'
NEW_MAPPING = 'new_clean.csv'

In [ ]:
OLD_DF = pd.read_csv(OLD_MAPPING, encoding='utf-8', low_memory=False)

In [14]:
#OLD_DF.count()

In [36]:

NEW_DF = pd.read_csv(NEW_MAPPING, encoding='utf-8', low_memory=False)

In [48]:

# Working_OLD_DataFrame
W_OLD_DF = OLD_DF[['project_id',
                   'bim360_account_id'
                   'business_unit_x',
                   'name_x',
                   'issues',
                   'users',
                   'issue_id',
                   'description',
                   'issue_type',
                   'status_x',
                   'company_assigned',
                   'job_number',
                   'identifier',
                   'created_at_x',
                   'due_date',
                   'closed_at',
                   'company_assigned',
                   'created_by',
                   'root_cause',
                   'area',
                   'location_detail',
                   'type',
                   'value',
                   'city',
                   'state_or_province',
                   'postal_code']].copy()

In [53]:
W_OLD_DF.head()
W_OLD_DF.rename(columns = {"name_x" : "name",
                           'business_unit_x' : 'business_unit',
                           'status_x' : 'status',
                           'created_at_x' : 'created_at'}, inplace=True)

In [54]:
W_OLD_DF.head()

,project_id,business_unit,name,issues,users,issue_id,description,issue_type,status,company_assigned,...,company_assigned,created_by,root_cause,area,location_detail,type,value,city,state_or_province,postal_code
0,01fa4f3a-9969-4069-a5b3-72fb630a0143,"Clayco>Distribution, Manufacturing, Process, F...",Gestamp,476,40,00bc0784-ed90-41fe-97d2-a71831f0e1dd,Remove nails around dock angles.,Punch List : Pre-Punch List,active,Concrete Strategies LLC,...,Concrete Strategies LLC,schniedersj@theljc.com,NaN,ELEV,NaN,Manufacturing / Factory,20549347.0,McCalla,AL,35111.0
1,01fa4f3a-9969-4069-a5b3-72fb630a0143,"Clayco>Distribution, Manufacturing, Process, F...",Gestamp,476,40,00d3a27e-8127-4dec-ba44-74da29274ac8,"Grind in a 3/4"" chamfer at all exterior corner...",Punch List : Pre-Punch List,active,Concrete Strategies LLC,...,Concrete Strategies LLC,schniedersj@theljc.com,Installation Contractor,ELEV,NaN,Manufacturing / Factory,20549347.0,McCalla,AL,35111.0
2,01fa4f3a-9969-4069-a5b3-72fb630a0143,"Clayco>Distribution, Manufacturing, Process, F...",Gestamp,476,40,00dfc82b-7220-4345-93ea-d0ff4ba06d03,Install missing switch and data outlets,Punch List : Architect Punch List,active,"Ace Electric, Inc.",...,"Ace Electric, Inc.",gallaghera@theljc.com,NaN,204 Training,NaN,Manufacturing / Factory,20549347.0,McCalla,AL,35111.0
3,01fa4f3a-9969-4069-a5b3-72fb630a0143,"Clayco>Distribution, Manufacturing, Process, F...",Gestamp,476,40,01a36881-f79f-4308-9fa1-55b48f7c58f2,Install cover plate on light switch,Punch List : Architect Punch List,active,"Ace Electric, Inc.",...,"Ace Electric, Inc.",gallaghera@theljc.com,NaN,117 RESTROOM,NaN,Manufacturing / Factory,20549347.0,McCalla,AL,35111.0
4,01fa4f3a-9969-4069-a5b3-72fb630a0143,"Clayco>Distribution, Manufacturing, Process, F...",Gestamp,476,40,01cebbe8-8a06-489f-a724-bb1b94844b0b,Install base,Punch List : Architect Punch List,active,Spectra Contract Flooring,...,Spectra Contract Flooring,gallaghera@theljc.com,NaN,118 OFFICE,NaN,Manufacturing / Factory,20549347.0,McCalla,AL,35111.0


In [58]:
NEW_DF.count()

Unnamed: 0                59084
project_id                59084
bim360_account_id_x       59084
name                      59084
start_date                58974
end_date                  58974
type                      59084
value                     57593
currency                  59084
status_x                  59084
job_number                57085
address_line1             42603
address_line2               883
city                      42841
state_or_province         42592
postal_code               42401
country                   59084
timezone                  59084
construction_type         39531
contract_type             39715
business_unit_id          57294
last_sign_in              59084
created_at_x              59084
issue_id                  59084
bim360_account_id_y       59084
display_id                59084
title                     59084
description               23200
type_id                   59084
subtype_id                59084
status_y                  59084
assignee

In [57]:

W_NEW_DF = NEW_DF[['project_id',
                   'bim360_account_id_x'
                   'business_unit_x',
                   'name',
                   'issues',
                   'users',
                   'issue_id',
                   'title',
                   'description',
                   'issue_type',
                   'status_x',
                   'company_assigned',
                   'job_number',
                   'identifier',
                   'created_at_x',
                   'due_date',
                   'closed_at',
                   'company_assigned',
                   'created_by',
                   'root_cause',
                   'area',
                   'location_detail',
                   'type',
                   'value',
                   'city',
                   'state_or_province',
                   'postal_code']].copy()

KeyError: "['users', 'issues', 'location_detail', 'identifier', 'company_assigned', 'issue_type', 'root_cause', 'bim360_account_id_xbusiness_unit_x', 'area'] not in index"